In [1]:
%cd ..
%pwd

/home/guilherme/scantensus-sdy


/home/guilherme/ai-env/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/home/guilherme/scantensus-sdy'

In [5]:
from pathlib import Path
from src.pressure_damping.label_processer import FirebaseCurveFetcher

fetcher = FirebaseCurveFetcher(Path('.firebase.json'))

fetcher.fetch('imp-coro-flow-inv')

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.